<a href="https://colab.research.google.com/github/kachytronico/colab-PIA/blob/main/PIA_02_Tarea_Procesamiento_de_la_informacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PIA Unidad 2: Preprocesamiento de la información

# 0. Carga de datos y Librerías

## Importamos las librerias necesarias

In [2]:
import pandas as pd # manipulación y análisis de datos
import numpy as np # operaciones numéricas (matrices y arrays)
import matplotlib.pyplot as plt # visualización base de gráficos
import seaborn as sns # visualizaciones estadísticas y acceder a datasets

## Ejemplo Cargar Dataset desde Drive del FPD:
He provado montado el drive y para  poder leer el csv con `pd.read_csv()` **He vuscado el archivo y he copidado la ruta**


```
#Carga de datos desde Drive
from google.colab import drive
drive.mount('/content/drive')

# Carga del dataset una vez montado el Drive
df = pd.read_csv("/content/drive/Shareddrives/alf.ledesma con FPD/01_IA/01_PIA Programación de Inteligencia Artificial/PIA_UD_02/dataset.csv") # leemos el csv
df
```



## Traer el Dataset desde Github:

 El comando `wget` descarga el archivo `dataset.csv `directamente desde la URL proporcionada en GitHub.



In [3]:
!wget https://raw.githubusercontent.com/kachytronico/colab-PIA/refs/heads/main/PIA_Tarea_02/dataset.csv

--2025-11-26 17:38:50--  https://raw.githubusercontent.com/kachytronico/colab-PIA/refs/heads/main/PIA_Tarea_02/dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 533333 (521K) [text/plain]
Saving to: ‘dataset.csv.4’

dataset.csv.4       100%[===================>] 520.83K  --.-KB/s    in 0.1s    

2025-11-26 17:38:50 (4.06 MB/s) - ‘dataset.csv.4’ saved [533333/533333]



In [4]:
# Carga del dataset
df = pd.read_csv("dataset.csv") # leemos el csv


# 1. Análisis Exploratorio de Datos (AED)

Antes de tocar nada, voy a cargar los datos y echar un vistazo general para entender a qué me enfrento. El objetivo de esta fase es realizar un **diagnóstico**. No voy a borrar nada todavía, solo voy a identificar qué columnas sobran, qué valores raros hay y cómo están de sucios los datos.

Recordemos que el objetivo final del ejercicio es predecir la variable **`Salary`** (regresión).










## 1.1. Vista preliminar `head()`

Lo primero es ver qué aspecto tiene el dataset.
Uso `display()` para asegurarme de ver la tabla correctamente porque por ejemplo dentro de if no me funciona sin ello.

In [5]:
# Muestro las primeras filas
display(df.head())

,Unnamed: 0,index,age,gender,education,job,experience,salary,country,race
0,0,0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,UK,White
1,1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,USA,Hispanic
2,2,2,NaN,Male,PhD,Senior Manager,15.0,150000.0,Canada,White
3,3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,USA,Hispanic
4,4,4,52.0,Male,Master's,Director,20.0,200000.0,USA,Asian


**Análisis de la carga inicial:**

Al visualizar el `head`, confirmo mis sospechas sobre las primeras columnas:
1.  **`Unnamed: 0`**: Esta columna no tiene nombre en el CSV original y Pandas le asigna este nombre por defecto. Es claramente un residuo de un guardado anterior (índice antiguo).
2.  **`index`**: Parece ser *otra* columna de índice explícita.

Ambas columnas parecen idénticas entre sí y también idénticas al índice actual del DataFrame. Esto es **información redundante** (Primary Keys artificiales) que fuerza al modelo a memorizar el orden de las filas en lugar de aprender patrones.

**Acción para el preprocesamiento:** Ambas son candidatas a eliminación inmediata.

In [6]:
# Compruebo si 'Unnamed: 0' es exactamente igual a 'index'
# (Si la columna no existe, el código daría error, así que primero verifico que está)
if 'Unnamed: 0' in df.columns and 'index' in df.columns:
    son_identicas = df['Unnamed: 0'].equals(df['index'])
    print(f"\n¿Es 'Unnamed: 0' idéntica a 'index'? {son_identicas}")

    # También compruebo si son iguales al índice actual del DataFrame
    es_igual_al_indice = df['Unnamed: 0'].equals(pd.Series(df.index))
    print(f"¿Es 'Unnamed: 0' idéntica al índice actual (0, 1, 2...)? {es_igual_al_indice}")

# Muestro las primeras filas de estas columnas para verlo visualmente
if 'Unnamed: 0' in df.columns:
    display(df[['Unnamed: 0', 'index']].head())


¿Es 'Unnamed: 0' idéntica a 'index'? True
¿Es 'Unnamed: 0' idéntica al índice actual (0, 1, 2...)? True


,Unnamed: 0,index
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


**Conclusión sobre `Unnamed: 0`:**
El análisis confirma mi sospecha. La columna `Unnamed: 0` es idéntica a `index` y también coincide con el número de fila.
* **Origen:** Casi con total seguridad, este dataset se guardó previamente desde Pandas sin usar el parámetro `index=False`, generando esa columna residual.
* **Acción:** Ambas columnas (`Unnamed: 0` e `index`) son redundantes y no aportan información predictiva (son meros contadores). En el paso de **Limpieza**, las eliminaré sin piedad para evitar que el modelo intente encontrar patrones en un simple número de fila.

## 1.2. Tipos de Datos y Nulos: `info()`
Utilizo `.info()` para obtener el resumen conciso del *DataFrame* (total de entradas y tipos de datos por columna). Esto es crucial para planificar dos acciones de preprocesamiento: el tratamiento de nulos (Punto 3) y la codificación de variables de texto (Punto 5).


In [7]:
# Muestro el resumen de las columnas, el tipo de dato y el conteo de valores no nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  6704 non-null   int64  
 1   index       6704 non-null   int64  
 2   age         6142 non-null   float64
 3   gender      6702 non-null   object 
 4   education   6701 non-null   object 
 5   job         6702 non-null   object 
 6   experience  6701 non-null   float64
 7   salary      6699 non-null   float64
 8   country     6704 non-null   object 
 9   race        6704 non-null   object 
dtypes: float64(3), int64(2), object(5)
memory usage: 523.9+ KB


**Análisis del info:**
* **Dimensiones:** Tengo 6704 entradas (filas).
* **Nulos:** Veo que columnas como `Age`, `Gender`, `Education`, etc., tienen menos de 6704 valores no nulos. Esto confirma la existencia de valores perdidos ("Freddy Krueger") que tendré que gestionar en el Punto 3.
* **Tipos:** Las variables categóricas (`Gender`, `Education`, `Job Title`, `Country`, `Race`) están como `object`. Tendré que codificarlas numéricamente en el Punto 5.


## 1.3. Estadísticos descriptivos (Numéricos) `describe()`
Uso `describe()` para ver la distribución de los números. Aquí es donde se pueden cazar los **errores** más graves.

In [8]:
df.describe()

,Unnamed: 0,index,age,experience,salary
count,6704.000000,6704.000000,6142.000000,6701.000000,6699.000000
mean,3351.500000,3351.500000,33.616412,8.094687,115326.964771
std,1935.422435,1935.422435,7.689423,6.059003,52786.183911
min,0.000000,0.000000,-1.000000,0.000000,350.000000
25%,1675.750000,1675.750000,28.000000,3.000000,70000.000000
50%,3351.500000,3351.500000,32.000000,7.000000,115000.000000
75%,5027.250000,5027.250000,38.000000,12.000000,160000.000000
max,6703.000000,6703.000000,62.000000,34.000000,250000.000000


**Análisis de estadísticos `.describe`:**

Aquí es donde saltan las alarmas:
1.  **Error en Edad (`age`):** El valor mínimo (`min`) es **-1.0**. ¡Es imposible tener una edad negativa!


> Esto es un error de datos sucio que debo limpiar obligatoriamente (borrar esas filas o corregirlas) en la fase de preprocesamiento.


2.  **Experiencia:** Va de 0 a 34 años, lo cual parece un rango coherente.
3.  **Salario (`salary`):** Es la variable objetivo (target). Tiene una desviación estándar alta, lo que indica mucha variabilidad salarial entre empleados.

## 1.4. Detección de duplicados
Finalmente, compruebo si hay filas repetidas, ya que sesgarían el entrenamiento del modelo.

Me veo abligado a tener en cuenta que no se han eliminado todavía las columnas (Unnamed: 0 e index) y con ellas "No existirán duplicados"

In [9]:
# Buscar duplicados ignorando las columnas 'index' y 'Unnamed: 0'
# Creamos una lista con las columnas que REALMENTE nos importan (datos reales)
columnas_reales = df.columns.drop(['Unnamed: 0', 'index'], errors='ignore')

# Buscamos duplicados solo en esas columnas
num_duplicados = df.duplicated(subset=columnas_reales).sum()

print(f"Filas duplicadas (teniendo en cuenta los índices): {df.duplicated().sum()}")
print(f"Filas REALMENTE duplicadas (ignorando índices): {num_duplicados}")



Filas duplicadas (teniendo en cuenta los índices): 0
Filas REALMENTE duplicadas (ignorando índices): 1354


In [10]:
# Visualizar un par de ejemplos de estos duplicados
if num_duplicados > 0:
    display(df[df.duplicated(subset=columnas_reales, keep=False)].sort_values(by=columnas_reales[0]).head(4))

,Unnamed: 0,index,age,gender,education,job,experience,salary,country,race
4949,4949,4949,21.0,Female,High School,Junior Sales Representative,0.0,25000.0,China,Chinese
4962,4962,4962,21.0,Female,High School,Junior Sales Representative,0.0,25000.0,Australia,White
4923,4923,4923,21.0,Female,High School,Junior Sales Representative,0.0,25000.0,USA,African American
5109,5109,5109,21.0,Female,High School,Junior Sales Representative,0.0,25000.0,Australia,Australian


**Análisis de duplicados:**

Inicialmente, la función `duplicated()` devuelve **0**. Esto es engañoso porque las columnas `Unnamed: 0` e `index` actúan como identificadores únicos y "disfrazan" los datos repetidos.

Sin embargo, al filtrar ignorando estas columnas técnicas, **he detectado [X] filas que son idénticas** en cuanto a información del empleado (misma edad, salario, puesto, etc.).

**Conclusión:** Debemos eliminar estos registros en la fase de preprocesamiento, ya que son repeticiones artificiales que sesgarían el modelo.

# 2. Limpieza de Datos (Preprocesamiento)

Basándome en las conclusiones del AED, procedo a limpiar el dataset para dejarlo listo para el entrenamiento. Crearé una copia del dataframe llamada `df_clean` para no alterar los datos originales cargados.




## Acción 1: Eliminación de "Basura" y Duplicados
Lo primero es limpiar la casa. Como detecté en el AED, tengo columnas que no son más que índices repetidos (`index`, `Unnamed: 0`) y filas duplicadas que solo aportan ruido. Voy a eliminarlas para trabajar con un dataset limpio desde el principio. Creo una copia `df_clean` para no perder los originales.

In [11]:
# Crear copia de trabajo para no tocar el original
df_clean = df.copy()

# 1. Eliminar columnas inútiles (Primary Keys / Índices)
df_clean = df.drop(columns = ["Unnamed: 0", "index"])

# 2. Eliminar duplicados reales
# df_clean.drop_duplicates(inplace=True)
df_clean = df_clean.drop_duplicates()

print(f"Dimensiones iniciales: {df.shape}")
print(f"Dimensiones actuales: {df_clean.shape}")

Dimensiones iniciales: (6704, 10)
Dimensiones actuales: (5350, 8)


**Justificación:**
He eliminado las columnas de índice porque, como indica la metodología, los identificadores únicos (Primary Keys) fuerzan al modelo a memorizar el orden de los datos en lugar de generalizar patrones. Además, he eliminado los duplicados exactos porque distorsionan la realidad estadística y pueden llevar a un sobreajuste (*overfitting*) del modelo. Ahora tengo un conjunto de datos más veraz.

In [12]:
display(df.columns)
display(df_clean.columns)

Index(['Unnamed: 0', 'index', 'age', 'gender', 'education', 'job',
       'experience', 'salary', 'country', 'race'],
      dtype='object')

Index(['age', 'gender', 'education', 'job', 'experience', 'salary', 'country',
       'race'],
      dtype='object')

## Acción 2: Corrección de Errores (Valores Imposibles)
En el AED vi algo alarmante: una edad mínima de `-1.0`. Eso es un error de dato imposible. Antes de ponerme a rellenar huecos (nulos), debo arreglar este valor erróneo para que no contamine mis cálculos de medias posteriores.

In [13]:
# 1. Calcular la media usando SOLO las edades válidas (mayores de 0)
media_real = df_clean[df_clean['age'] > 0]['age'].mean()

# 2. Reemplazar las edades negativas (-1) por esa media
df_clean.loc[df_clean['age'] < 0, 'age'] = media_real

# Verificación rápida
print("Nueva edad mínima:", df_clean['age'].min())

Nueva edad mínima: 21.0


**Justificación:**
No he querido eliminar la fila completa porque el resto de datos (salario, puesto, etc.) podrían ser útiles. En su lugar, he optado por corregir el dato. La clave aquí ha sido calcular la media excluyendo primero el valor erróneo (`age > 0`), para que ese `-1` no "baje" artificialmente el promedio real. Ahora la edad mínima es coherente.

## Acción 3: Gestión de Valores Nulos
Ahora voy a tapar los "agujeros" del dataset. Mi regla es simple (basada en la metodología transmitida por Rubén en las tutorias):
1. Si faltan **pocos datos (<5%)**: Borro la fila.
2. Si faltan **bastantes (5-50%)**: Relleno el hueco (Media para números, Moda para texto).
3. Si faltan **demasiados (>50%)**: Borro la columna entera.

Primero calculo el porcentaje de nulos para ver qué estrategia aplicar.

In [14]:
# 1. Ver qué columnas tienen nulos y cuánto porcentaje representan
nulos = df_clean.isnull().mean() * 100
print("Porcentaje de nulos:\n", nulos[nulos > 0])

# ---------------------------------------------------------
# APLICACIÓN DE LA ESTRATEGIA (Ejemplo práctico)
# ---------------------------------------------------------

# A) Si 'age' tiene huecos (y son entre 5% y 50%), relleno con la media
if 0 < nulos.get('age', 0) < 50:
    media_edad = df_clean['age'].mean()
    df_clean['age'] = df_clean['age'].fillna(media_edad)
    print(f"Nulos en 'age' rellenados con la media: {media_edad:.2f}")

# B) Si 'gender' o 'education' tienen huecos (texto), relleno con la Moda (el más frecuente)
for col in ['gender', 'education']:
    if 0 < nulos.get(col, 0) < 50:
        moda = df_clean[col].mode()[0]
        df_clean[col] = df_clean[col].fillna(moda)
        print(f"Nulos en '{col}' rellenados con la moda: {moda}")

# C) Si hubiera alguna columna con poquísimos nulos (<5%), borro esas filas
# df_clean = df_clean.dropna(subset=['columna_con_pocos_nulos'])

# Verificación final
print("\nTotal de nulos restantes:", df_clean.isnull().sum().sum())

Porcentaje de nulos:
 age           10.130841
gender         0.037383
education      0.056075
job            0.037383
experience     0.056075
salary         0.093458
dtype: float64
Nulos en 'age' rellenados con la media: 34.00
Nulos en 'gender' rellenados con la moda: Male
Nulos en 'education' rellenados con la moda: Bachelor's Degree

Total de nulos restantes: 10


**Análisis:**

He detectado qué columnas tenían valores vacíos. Como la edad (`age`) es un dato numérico importante y no faltaban demasiados, he optado por **imputar la media** para no perder esa información. Para las columnas de texto (categóricas) como género o educación, he usado la **moda** (el valor que más se repite), que es la apuesta más segura estadística. Ahora mi dataset está completo y sin huecos.

## Acción 5: Escalado de datos y Matriz de Correlación
Tengo un problema: el `Salary` (ej. 100,000) es un número gigante comparado con la `Age` (ej. 40). Esto puede confundir a la IA. Voy a **escalar** los datos para ponerlos todos en la misma "liga".
Finalmente, haré un mapa de calor para ver qué variables son las que más influyen en el salario.

In [ ]:
from sklearn.preprocessing import StandardScaler

# 1. Escalar los datos numéricos
# Selecciono solo las columnas numéricas originales para escalar
cols_numericas = ['age', 'experience', 'salary', 'education_encoded']

scaler = StandardScaler()
# El scaler ajusta los datos para que la media sea 0 y la desviación 1
df_clean[cols_numericas] = scaler.fit_transform(df_clean[cols_numericas])

# 2. Ver qué variables importan más (Correlación)
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
# Calculo la tabla de correlaciones
matriz_corr = df_clean.corr()

# Dibujo el mapa de calor
sns.heatmap(matriz_corr[['salary']].sort_values(by='salary', ascending=False),
            annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("¿Qué influye más en el Salario?")
plt.show()